# European vanilla option pricing

This notebook aims to be a simple tool for pricing european options through a Monte Carlo simulation. The code is based on that developed in **Joshi, Mark S. (2008) *C++ Design Patterns and Derivatives Pricing*. New York: Cambridge University Press**, though adapted to Python language.

**Disclaimer: I am not the owner of the piece of code shown below. For more information, refer to the book mentioned before.**

European options are derivatives that, upon payment of a premium, give the holder the right, but not the obligation, to buy or sell the underlying at maturity of the option.

The price evolution of a stock can be seen as an Itô process ruled by the following stochastic equation: $$dS_t = \mu S_tdt + \sigma S_tdz$$ 

where $\mu$ is the stock's expected rate of return and $\sigma$ is the volatility of the stock price. Now, using Itô's lemma on the previous expression,

$$G=G\left(S_t,t\right)$$

$$ dG = \left[\frac{\partial{G}}{\partial{S_t}}\mu S_t+\frac{\partial{G}}{\partial{t}}+\frac{1}{2}\frac{\partial{}}{\partial{S_t}}\left(\frac{\partial{G}}{\partial{S_t}}\right)\sigma^2 S_t^2 \right]dt+\left(\frac{\partial{G}}{\partial{S_t}}\sigma S_t \right)dz$$

and assuming a lognormal distribution for the stock price,

$$ G\left(S_t, t\right)=\ln S_t$$

$$dG=\left[\frac{1}{S_t}\mu S_t+\frac{1}{2}\left(\frac{-1}{S_t^2}\right)\sigma^2 S_t^2\right]dt+\frac{1}{S_t}\sigma S_tdz$$

$$dG=\left(\mu-\frac{1}{2}\sigma^2\right)dt+\sigma dz$$

$$\ln S_t-\ln S_0=\left(\mu-\frac{1}{2}\sigma^2\right)T+\sigma \varepsilon \sqrt{T}$$
where $\varepsilon \sim N\left(0,1\right)$

We can rewrite this expression as:
$$S_t=S_0e^{\left(\mu-\frac{1}{2}\sigma^2\right)T+\sigma \varepsilon \sqrt{T}}$$

Assuming that the expected rate of return $\mu$ is the risk-free rate $r$ and under the risk-neutral hypothesis, the value of a vanilla option is therefore equal to the discounted expected payoff:
$$e^{-rT}E \left[f\left(S_0e^{\left(r-\frac{1}{2}\sigma^2\right)T+\sigma \varepsilon \sqrt{T}}\right)\right] $$
and $f$ is $\left[S_t-K\right]^+$ in case of a call and $\left[K-S_t\right]^+$ if it's a put.

With the Monte Carlo algorithm, we can get the price of an option by drawing a random number from an $N(0,1)$ distribution.

In [1]:
from random import random
from math import exp, sqrt, log

In [2]:
def GetOneGaussianBySummation():
    result = 0
    for j in range(12):
        result += random()
    
    result -= 6
    return result

In [3]:
def GetOneGaussianByBoxMuller():
    sizeSquared = 1
    while sizeSquared >= 1.0:
        x = 2.0*random() - 1
        y = 2.0*random() - 1
        sizeSquared = x*x + y*y
    
    result = x*sqrt(-2*log(sizeSquared)/sizeSquared)
    return result

In [4]:
def SimpleMonteCarlo1(Expiry, Strike, Spot, Vol, r, NumberOfPaths, OptionType):
    variance = Vol*Vol*Expiry
    rootVariance = sqrt(variance)
    itoCorrection = -0.5*variance
    
    movedSpot = Spot*exp(r*Expiry + itoCorrection)
    runningSum = 0
    
    if OptionType == 'call':
        for i in range(NumberOfPaths):
            thisGaussian = GetOneGaussianByBoxMuller()
            thisSpot = movedSpot*exp(rootVariance*thisGaussian)
            thisPayoff = thisSpot - Strike
            if(thisPayoff<0):
                thisPayoff=0
            runningSum += thisPayoff

    elif OptionType == 'put':
        for i in range(NumberOfPaths):
            thisGaussian = GetOneGaussianByBoxMuller()
            thisSpot = movedSpot*exp(rootVariance*thisGaussian)
            thisPayoff = Strike - thisSpot
            if(thisPayoff<0):
                thisPayoff=0
            runningSum += thisPayoff
    
    mean = runningSum / NumberOfPaths
    mean *= exp(-r*Expiry)
    return mean

In [5]:
Expiry = 1
Strike = 100
Spot = 100
Volatility = 0.2
r = 0.05
NumberOfIterations = 1000000
CallOption = "call"
PutOption = "put"

callPrice = SimpleMonteCarlo1(Expiry, Strike, Spot, Volatility, r, NumberOfIterations, CallOption)
print(f'Call price: {round(callPrice, 5)}, for {NumberOfIterations} paths')
putPrice = SimpleMonteCarlo1(Expiry, Strike, Spot, Volatility, r, NumberOfIterations, PutOption)
print(f'Put price: {round(putPrice, 5)}, for {NumberOfIterations} paths')

Call price: 10.44869, for 1000000 paths
Put price: 5.58144, for 1000000 paths
